In [1]:
import pandas as pd
import random
import math
#from penalty_cost import calculate_penalty_cost
import copy
df=pd.read_excel('Line Production December 2023.xlsx')

#  %run penalty_cost.ipynb

In [2]:
def calculate_penalty_cost(df, solution):
    total_penalty_cost = 0

    for line, products in solution.items():

        #initial values 
        line_penalty_cost = 0  
        current_time = 0  

        for product in products:
            #get the data of the specific product that's in the current line 
            product_row = df[df['Product'] == product]

            if not product_row.empty:

                # the products processing time on the line it's being processed on 
                processing_time = product_row[line].values[0]
                # product deadline 
                deadline = product_row['deadline'].values[0]

                # how late the product is 
                lateness = current_time + processing_time - deadline

                #check if the product is late and if yes what the cost is     
                if lateness > 0:
                    penalty_cost = lateness * product_row['penalty cost'].values[0]
                    #add the penalty cost to total cost of that line
                    line_penalty_cost += penalty_cost

                #change current time to that after processing the product 
                current_time += processing_time

        # add penalty cost of everyline together 
        total_penalty_cost += line_penalty_cost

    return total_penalty_cost

In [28]:
initial_temperature = 10
cooling_rate = 0.995
iterations_per_temperature = 100

# Simulated annealing function
def simulated_annealing(df, schedule, temperature, cooling_rate, iterations_per_temp):
    
    current_schedule = copy.deepcopy(schedule)
    best_schedule = copy.deepcopy(schedule)
    current_cost = calculate_penalty_cost(df, current_schedule)
    best_cost = current_cost

    while temperature > 0.1:
        for k in range(iterations_per_temp):
            # Generate a neighboring solution by swapping two random cities
            i, j = random.sample(list(current_schedule.keys()), 2)

            # Use indexing to find items in the random lines and replace them in the original dictionary
            item_index_i = random.randint(0, len(current_schedule[i]) - 1)
            item_index_j = random.randint(0, len(current_schedule[j]) - 1)

            # Swap the items
            neighbor_schedule = copy.deepcopy(current_schedule)
            neighbor_schedule[i][item_index_i], neighbor_schedule[j][item_index_j] = neighbor_schedule[j][item_index_j], neighbor_schedule[i][item_index_i]
 
            #neighbor_schedule = current_schedule[:]
            neighbor_cost = calculate_penalty_cost(df, neighbor_schedule)

            # Decide whether to accept the neighbor solution
            delta = neighbor_cost - current_cost
            if delta < 0 or random.random() < math.exp(-delta / temperature):
                current_schedule = neighbor_schedule
                current_cost = neighbor_cost

                # Update the best solution if necessary
                if current_cost < best_cost:
                    best_schedule = current_schedule
                    best_cost = current_cost

        # Cool the temperature
        temperature *= cooling_rate

    return best_schedule, best_cost

initial_solution2 = {"L1": ['P26', 'P17', 'P18', 'P20'],
"L2": ['P10', 'P7', 'P27', 'P19', 'P12'],
"L3": ['P13', 'P14', 'P22', 'P15'],
"L4": ['P11', 'P9', 'P25'],
"L5": ['P23', 'P16', 'P21'],
"L6": ['P2', 'P5', 'P8', 'P1'],
"L7": ['P6', 'P4', 'P24', 'P3']}

initial_solution = {
    'L1': ['P26', 'P9', 'P24', 'P1'],
    'L2': ['P7', 'P27', 'P10', 'P8'],
    'L3': ['P13', 'P22', 'P25', 'P14'],
    'L4': ['P11', 'P23', 'P2'],
    'L5': ['P16', 'P21', 'P19', 'P18'],
    'L6': ['P5', 'P20', 'P15'],
    'L7': ['P6', 'P4', 'P17', 'P12', 'P3']
}
Solution1 = {'L1': ['P26', 'P6', 'P9', 'P24'],
 'L2': ['P7', 'P16', 'P27', 'P15', 'P10'],
 'L3': ['P13', 'P25'],
 'L4': ['P4', 'P8'],
 'L5': ['P21', 'P18', 'P3'],
 'L6': ['P11', 'P5', 'P23', 'P19', 'P20', 'P2', 'P14'],
 'L7': ['P22', 'P17', 'P1', 'P12']}

december_solution = {
    'L1': ['P5', 'P25'],
    'L2': ['P12', 'P2', 'P1', 'P8', 'P19', 'P16', 'P24'],
    'L3': ['P13', 'P14', 'P22', 'P15', 'P6', 'P4', 'P18', 'P20', 'P26'],
    'L4': ['P11', 'P9', 'P10', 'P7', 'P17'],
    'L5': ['P21'],
    'L6': ['P27', 'P3'],
    'L7': ['P23']
}




best_schedule, best_cost = simulated_annealing(df, december_solution, initial_temperature, cooling_rate, iterations_per_temperature)


print("Initial Solution:")
for line, products in december_solution.items():
    print(f"{line}: {', '.join(products)}")
print("Initial Penalty Cost:", calculate_penalty_cost(df, december_solution))
print("")
print("Final Best Solution:")
for line, products in best_schedule.items():
    print(f"{line}: {', '.join(products)}")
print("Final Best Penalty Cost:", best_cost)


Initial Solution:
L1: P5, P25
L2: P12, P2, P1, P8, P19, P16, P24
L3: P13, P14, P22, P15, P6, P4, P18, P20, P26
L4: P11, P9, P10, P7, P17
L5: P21
L6: P27, P3
L7: P23
Initial Penalty Cost: 142218

Final Best Solution:
L1: P9, P6
L2: P16, P7, P27, P19, P24, P15, P20
L3: P13, P25, P22, P12, P1, P2, P18, P3, P14
L4: P11, P4, P10, P8, P23
L5: P17
L6: P26, P5
L7: P21
Final Best Penalty Cost: 15034
